<a href="https://colab.research.google.com/github/sgmurphy00/TOOLS1/blob/google_collab_environment/dev_acgan_general_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Baseline Packages of Colab
!pip freeze > requirements_v1.txt


In [ ]:
'''
Author: Paul Scheidt
Date: 10-23-2024
Description:
This Python Script will Run intial machine learning image classifier. Then
run basic ACGAN algorithm
'''

In [16]:
# Import Packages for Machine Learning Image Classifier for Sat Data
from google.colab import drive  # Required to path to google drive where data is stored
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import time
from IPython import display
import tfgraphviz as tfg
from visualkeras import layered_view
print(tf.__version__)

2.17.0


In [6]:
# Connecting to Deep Stat Database
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
DATA_PATH = '/content/drive/My Drive/DEEPSAT6/'

In [8]:
# Initializing training and valiadation Data Paths
train_data_path = DATA_PATH + '/X_train_sat6.csv'
train_label_path = DATA_PATH + 'y_train_sat6.csv'
test_data_path = DATA_PATH + 'X_test_sat6.csv'
test_label_path = DATA_PATH + 'y_test_sat6.csv'

In [9]:
# Storing data into pandas dataframes
x_train = pd.read_csv(train_data_path, header=None, dtype='uint8')
y_train = pd.read_csv(train_label_path, header=None, dtype='uint8')
x_test = pd.read_csv(test_data_path, header=None, dtype='uint8')
y_test = pd.read_csv(test_label_path, header=None, dtype='uint8')

In [10]:
# Investigating Shape of Pandas DataFrame
print("Train data shape: ", x_train.shape)
print("Train label shape: ", y_train.shape)
print("Test data shape: ", x_test.shape)
print("Test label shape: ", y_test.shape)

Train data shape:  (324000, 3136)
Train label shape:  (324000, 6)
Test data shape:  (81000, 3136)
Test label shape:  (81000, 6)


In [11]:
# Restructuring Training and Test Images to be 3 layers, RGB
x_train = x_train.values.reshape([-1,28,28,3])
x_test = x_test.values.reshape([-1,28,28,3])

x_train.shape, x_test.shape

((432000, 28, 28, 3), (108000, 28, 28, 3))

In [33]:
# Building Generator Model based on Architecture from Conditional Image Synthesis with Auxiliary Classifier GANs, Table 1
def generator_model():
  input = tf.keras.Input(shape=(110,))
  labels_class = tf.keras.Input(shape=(6,))  # 6 different classes in data set

  # Cpmcatenate takes as input a list of tensors, all of the same shape except for the concatenation axis, and returns a single tensor that is the concatenation of all inputs.
  x = tf.keras.layers.Concatenate(axis=-1)([input,labels_class])

  # Linear Layer:
  x = tf.keras.layers.Dense(7*7*256,input_shape=(110,))(x)  # No Activation Defined, defaults to Linear
  x = tf.keras.layers.Reshape((7,7,256))(x)
  x = tf.keras.layers.Conv2DTranspose(128,(5,5),strides=(1,1),padding='same',activation='relu',name='Transposed Convolution-1')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv2DTranspose(128,(5,5),strides=(1,1),padding='same',activation='relu',name='Transposed Convolution-2')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv2DTranspose(64,(5,5),strides=(2,2),padding='same',activation='relu',name='Transposed Convolution-3')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv2DTranspose(1,(5,5),strides=(2,2),padding='same',activation='tanh',name='Transposed Convolution-4')(x)
  return tf.keras.Model(inputs = [input, labels_class], outputs = x)

generator = generator_model()
generator.summary()



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_22            │ (None, 110)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_23            │ (None, 6)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_11            │ (None, 116)            │              0 │ input_layer_22[0][0],  │
│ (Concatenate)             │                        │                │ input_layer_23[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 12544)          │      1,467,648 │ concatenate_11[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_9 (Reshape)       │ (None, 7, 7, 256)      │              0 │ dense_10[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Transposed Convolution-1  │ (None, 7, 7, 128)      │        819,328 │ reshape_9[0][0]        │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_22    │ (None, 7, 7, 128)      │            512 │ Transposed             │
│ (BatchNormalization)      │                        │                │ Convolution-1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Transposed Convolution-2  │ (None, 7, 7, 128)      │        409,728 │ batch_normalization_2… │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_23    │ (None, 7, 7, 128)      │            512 │ Transposed             │
│ (BatchNormalization)      │                        │                │ Convolution-2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Transposed Convolution-3  │ (None, 14, 14, 64)     │        204,864 │ batch_normalization_2… │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_24    │ (None, 14, 14, 64)     │            256 │ Transposed             │
│ (BatchNormalization)      │                        │                │ Convolution-3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Transposed Convolution-4  │ (None, 28, 28, 1)      │          1,601 │ batch_normalization_2… │
│ (Conv2DTranspose)         │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,904,449 (11.08 MB)

 Trainable params: 2,903,809 (11.08 MB)

 Non-trainable params: 640 (2.50 KB)

/usr/local/lib/python3.10/dist-packages/visualkeras/layered.py:86: UserWarning: The legend_text_spacing_offset parameter is deprecated and will be removed in a future release.
  warnings.warn("The legend_text_spacing_offset parameter is deprecated and will be removed in a future release.")
